In [5]:
import json
import numpy as np
import dicom
import os
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics
import random
import volume_final_working as vfw
import cv2

In [6]:
v_dir = '/masvol/output/dsb/volume/'
dirs = ['3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2',
    '3combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2',
    '3combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2',
        '3combined_1_3_0_176_CLAHE_augx_bce1_crERDS2',
       '3combined_1_3_0_176_aug_drop_dice_crERDS2']
# dirs = ['3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2']
#        '3combined_1_3_0_176_aug_drop_dice_cr_v2',
#        '3combined_1_3_0_176_CLAHE_augx_bce1_crNCR',
#        '3combined_1_3_0_176_CLAHE_augx_bce1_crERDS2',
#        '3combined_1_3_0_176_aug_drop_dice_update_cr',
#        '3combined_1_3_0_176_CLAHE_augx_bn_bce1_limit_crERDS2']

In [9]:
def volDict(paths, diff, vtype='orig',STway='sub', ty=None, rls = None, stcheck = None):
    v_dict = {}
    no_data = []
    print (paths)
    for i in range(len(paths)):
#         print (paths[i], i)
        d = paths[i]
#         print (d[1:])
        splits = d.split('_')
#         print (splits)
        if ('176') in d:
            imgsize = 176
#             print ('IMAGE SIZE 176')
        elif ('256') in d:
            imgsize = 256
        v = vfw.computeVolume(v_dir+splits[1]+'/'+paths[i]+'/',imgsize, vtype=vtype,STway=STway, ty=ty, rls = rls, stcheck = stcheck)
        print (v_dir+splits[1]+'/'+paths[i]+'/')
#         print (v)
        if len(v.keys()) > 0:
            train_df, validate_df, all_df = vfw.create_df(v)
            all_no_outliers = vfw.removeOutliers(all_df,v, data=None)
            print (all_no_outliers.columns)
            test = vfw.removeLowEstimates(all_no_outliers, data = None)
            print (test.columns)
            all_no_out_noVol = vfw.removeNoVolume(test)
            sys, dia, ef = vfw.compute_rmse(all_no_out_noVol, 'all')
#             v1_sys_low, v1_sys_high,sys_lows,sys_highs = vole.
            v1_sys_low, v1_sys_high,sys_lows,sys_highs = vfw.estimateCheck(all_no_out_noVol, actual='Systole',predicted='Systole_P',dev=diff)
            v1_dia_low, v1_dia_high,dia_lows, dia_highs = vfw.estimateCheck(all_no_out_noVol, actual='Diastole',predicted='Diastole_P',dev=diff)
            top_sys = vfw.getbest(all_no_out_noVol, actual='Systole',predicted='Systole_P',dev=1)
            top_dia = vfw.getbest(all_no_out_noVol, actual='Diastole',predicted='Diastole_P',dev=1)
#             print (v_dict)
            r = '_'
            s= '_'
            if rls is not None:
                r = '_rls'
            if stcheck == 'Slice_ST':
                s = '_SliceST'
            if stcheck == 'Large_diff':
                s = '_LargeDiff'
            vol_model = vtype + r + s
            print (vol_model)
            if d in v_dict:
#                 print ('****************'+d)
                v_dict[d[1:]].update({'Vol Method': vol_model,
                                   'Systole RMSE': sys,
                                  'Diastole RMSE': dia,
                                  'EF RMSE': ef,
                                  '% Sys Low': v1_sys_low,
                                  '% Sys High': v1_sys_high,
                                  '% Dia Low': v1_dia_low, 
                                  '% Dia High': v1_dia_high, 
                                  'Sys Lows': sys_lows,
                                  'Sys Highs': sys_highs,
                                  'Dia Lows': dia_lows,
                                  'Dia Highs': dia_highs,
                                 'Top Sys': top_sys,
                                 'Top Dia': top_dia})
            else:
#                 print ('*************** ELSE')
                v_dict[d[1:]]=({'Vol Method': vol_model,
                             'Systole RMSE': sys,
                                  'Diastole RMSE': dia,
                                  'EF RMSE': ef,
                                  '% Sys Low': v1_sys_low,
                                  '% Sys High': v1_sys_high,
                                  '% Dia Low': v1_dia_low, 
                                  '% Dia High': v1_dia_high,
                                    'Sys Lows': sys_lows,
                                  'Sys Highs': sys_highs,
                                  'Dia Lows': dia_lows,
                                  'Dia Highs': dia_highs,
                           'Top Sys': top_sys,
                           'Top Dia': top_dia})
        else:
            no_data.append(paths[i])
#     print (v_dict)
    return (v_dict, no_data)

In [10]:
volume_f,no_data = volDict(dirs, 50, vtype= 'orig', rls=None, stcheck = None)
vol_f=pd.DataFrame.from_dict(volume_f, orient='index')

['3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_bce1_crERDS2', '3combined_1_3_0_176_aug_drop_dice_crERDS2']
/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2/
['599', '595']
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [63.6 140.2 0.0 0.0 '015Y' 15 'Y' 'F' 3 63.6 140.2 nan 0.5463623395149786]
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [130.4 278.7 6.155000209696996 67.310002813146 '018Y' 18 'Y' 'M' 3
 124.24499979030301 211.38999718685398 0.9085574215947754
 0.

/masvol/heartsmart/Volume/volume_final_working.py:530: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(few_slices[i], 'Systole_P', new_sys, takeable = False)
/masvol/heartsmart/Volume/volume_final_working.py:531: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(few_slices[i], 'Diastole_P', new_dia, takeable = False)
/masvol/heartsmart/Volume/volume_final_working.py:533: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(few_slices[i], 'EF_P', new_ef, takeable = False)
/masvol/heartsmart/Volume/volume_final_working.py:554: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(few_slices[i], 'Systole_P', new_sys, takeable = False)
/masv

/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2/
['599', '595']
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [63.6 140.2 0.0 7.659999708339505 '015Y' 15 'Y' 'F' 3 63.6
 132.54000029166048 1.0 0.5463623395149786]
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [130.4 278.7 14.985000604002158 65.71500276143404 '018Y' 18 'Y' 'M' 3
 115.41499939599785 212.98499723856594 0.7719698702836186
 0.532113383566559]
[array([63.6, 140.2, 0.0, 7.659999708339505, '015Y', 15, 'Y', 'F', 3, 63.6,
       132.54000029166048, 1.0, 0.5463623395149786], dtype=object), array([130.4, 278.7, 14.985000604002158, 65.715002761

/masvol/heartsmart/Volume/volume_final_working.py:748: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'Diastole_P', new_vol, takeable=False)
/masvol/heartsmart/Volume/volume_final_working.py:750: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'EF_P', ef_p, takeable = False)


/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2/
['599', '595']
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [63.6 140.2 0.0 23.144999436545287 '015Y' 15 'Y' 'F' 3 63.6
 117.0550005634547 1.0 0.5463623395149786]
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [130.4 278.7 31.31500128854159 67.38000282055626 '018Y' 18 'Y' 'M' 3
 99.08499871145841 211.31999717944373 0.5352478483573463 0.532113383566559]
[array([63.6, 140.2, 0.0, 23.144999436545287, '015Y', 15, 'Y', 'F', 3,
       63.6, 117.0550005634547, 1.0, 0.5463623395149786], dtype=object), array([130.4, 278.7, 31.31500128854159, 67.380002820556

/masvol/heartsmart/Volume/volume_final_working.py:707: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'Systole_P', new_vol, takeable=False)
/masvol/heartsmart/Volume/volume_final_working.py:709: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'EF_P', ef_p, takeable = False)


/masvol/output/dsb/volume/1/3combined_1_3_0_176_aug_drop_dice_crERDS2/
['599', '595']
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [63.6 140.2 0.0 12.779999753841203 '015Y' 15 'Y' 'F' 3 63.6
 127.42000024615879 1.0 0.5463623395149786]
<class 'pandas.core.series.Series'> Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object') [130.4 278.7 16.32500066975675 68.32000284972085 '018Y' 18 'Y' 'M' 3
 114.07499933024326 210.37999715027914 0.7610509369318118
 0.532113383566559]
[array([63.6, 140.2, 0.0, 12.779999753841203, '015Y', 15, 'Y', 'F', 3,
       63.6, 127.42000024615879, 1.0, 0.5463623395149786], dtype=object), array([130.4, 278.7, 16.32500066975675, 68.32000284972085,

In [12]:
vol_f[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')

,Systole RMSE,Diastole RMSE,EF RMSE
combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2,16.26,14.90,19.0
combined_1_3_0_176_CLAHE_augx_bce1_crERDS2,16.57,16.73,13.0
combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2,13.15,17.30,10.0
combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2,15.17,17.63,9.0
combined_1_3_0_176_aug_drop_dice_crERDS2,13.55,18.72,8.0


In [9]:
vol_f[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')
# vol_f['Vol_Method'] = 'orig'

,Systole RMSE,Diastole RMSE,EF RMSE
combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2,16.14,14.46,19.0
combined_1_3_0_176_CLAHE_augx_bce1_crERDS2,16.45,16.34,13.0
combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2,12.99,16.93,10.0
combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2,15.04,17.26,9.0
combined_1_3_0_176_aug_drop_dice_crERDS2,13.40,18.38,8.0


In [10]:
volume_f,no_data = volDict(dirs, 50, vtype= 'zeros', rls=None, stcheck = None)
vol_f_1=pd.DataFrame.from_dict(volume_f, orient='index')

['3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_bce1_crERDS2', '3combined_1_3_0_176_aug_drop_dice_crERDS2']
/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2/
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
all :  Systole RMSE: 14.36 ml  Diastole RMSE: '14.46 ml  EF RMSE: 18.00 %
Systole : % Low Estima

/masvol/heartsmart/Volume/volume_final_working.py:666: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'Systole_P', new_vol, takeable=False)
/masvol/heartsmart/Volume/volume_final_working.py:667: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'EF_P', ef_p, takeable = False)
/masvol/heartsmart/Volume/volume_final_working.py:674: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'Systole_P', new_vol, takeable=False)
/masvol/heartsmart/Volume/volume_final_working.py:675: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'EF_P', ef_p, takeable = False)


/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2/
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
all :  Systole RMSE: 14.38 ml  Diastole RMSE: '17.26 ml  EF RMSE: 8.00 %
Systole : % Low Estimate: 1.0 % Correct Estimate:  37.0 % High Estimate:  0.0
Lows:  [82, 119, 242, 269, 274, 437, 478, 579]
Highs:  [429]
Diastole : % Low Estimate: 0.0 % Correct Estimate:  28.999999999999996 % High Estimate:  1.0
Lows:  [269, 412, 545]
Highs:  [362,

/masvol/heartsmart/Volume/volume_final_working.py:720: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'Diastole_P', new_vol, takeable=False)
/masvol/heartsmart/Volume/volume_final_working.py:721: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'EF_P', ef_p, takeable = False)


/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2/
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
all :  Systole RMSE: 12.40 ml  Diastole RMSE: '16.93 ml  EF RMSE: 8.00 %
Systole : % Low Estimate: 0.0 % Correct Estimate:  36.0 % High Estimate:  0.0
Lows:  [242, 269]
Highs:  [429]
Diastole : % Low Estimate: 0.0 % Correct Estimate:  28.000000000000004 % High Estimate:  1.0
Lows:  [279]
Highs:  [362, 429, 436, 561, 579, 643, 684]
zeros__

In [11]:
vol_f_1[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')
# vol_f_1['Vol_Method'] = 'zeros'

,Systole RMSE,Diastole RMSE,EF RMSE
combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2,14.36,14.46,18.0
combined_1_3_0_176_CLAHE_augx_bce1_crERDS2,13.58,16.34,9.0
combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2,12.40,16.93,8.0
combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2,14.38,17.26,8.0
combined_1_3_0_176_aug_drop_dice_crERDS2,12.75,18.38,7.0


In [12]:
volume_f,no_data = volDict(dirs, 50, vtype= 'zeros', rls='Yes', stcheck = None)
vol_f_2=pd.DataFrame.from_dict(volume_f, orient='index')

['3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2', '3combined_1_3_0_176_CLAHE_augx_bce1_crERDS2', '3combined_1_3_0_176_aug_drop_dice_crERDS2']
/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2/
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
all :  Systole RMSE: 14.45 ml  Diastole RMSE: '14.67 ml  EF RMSE: 17.00 %
Systole : % Low Estima

/masvol/heartsmart/Volume/volume_final_working.py:666: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'Systole_P', new_vol, takeable=False)
/masvol/heartsmart/Volume/volume_final_working.py:667: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'EF_P', ef_p, takeable = False)
/masvol/heartsmart/Volume/volume_final_working.py:674: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'Systole_P', new_vol, takeable=False)
/masvol/heartsmart/Volume/volume_final_working.py:675: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'EF_P', ef_p, takeable = False)
/masvol/heartsmart/Volume/volume_final_working.py:720: FutureWarning

/masvol/output/dsb/volume/1/3combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2/
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
Index(['Systole', 'Diastole', 'Systole_P', 'Diastole_P', 'Age', 'Age_Int',
       'Age Unit', 'Gender', 'NumSlices', 'Systole_diff', 'Diastole_diff',
       'EF_P', 'EF'],
      dtype='object')
all :  Systole RMSE: 14.61 ml  Diastole RMSE: '15.76 ml  EF RMSE: 9.00 %
Systole : % Low Estimate: 1.0 % Correct Estimate:  37.0 % High Estimate:  0.0
Lows:  [82, 119, 242, 269, 274, 437, 478, 579]
Highs:  [429]
Diastole : % Low Estimate: 1.0 % Correct Estimate:  28.999999999999996 % High Estimate:  1.0
Lows:  [172, 269, 279, 412, 545]
Hig

In [13]:
vol_f_2[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')
# vol_f_2['Vol_Method'] = 'zeros_rls'

,Systole RMSE,Diastole RMSE,EF RMSE
combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2,14.45,14.67,17.0
combined_1_3_0_176_CLAHE_augx_bce1_crERDS2,13.69,14.82,9.0
combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2,12.44,14.97,10.0
combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2,14.61,15.76,9.0
combined_1_3_0_176_aug_drop_dice_crERDS2,12.90,15.79,7.0


In [ ]:
# volume_f,no_data = volDict(dirs, 50, vtype= 'zeros', rls=None, stcheck = 'Slice_ST')
# vol_f_3=pd.DataFrame.from_dict(volume_f, orient='index')

In [ ]:
# vol_f_3[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')
# # vol_f_3['Vol_Method'] = 'zeros_SliceST'

In [ ]:
# volume_f,no_data = volDict(dirs, 50, vtype= 'zeros', rls='Yes', stcheck = 'Slice_ST')
# vol_f_4=pd.DataFrame.from_dict(volume_f, orient='index')

In [ ]:
# vol_f_4[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')
# vol_f_4['Vol_Method'] = 'zeros_rls_SliceST'

In [ ]:
# volume_f,no_data = volDict(dirs, 50, vtype= 'zeros', rls='Yes', stcheck = 'Large_Diff')
# vol_f_5=pd.DataFrame.from_dict(volume_f, orient='index')

In [ ]:
# vol_f_5[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')
# vol_f_5['Vol_Method'] = 'zeros_rls_Large_Diff'

In [ ]:
# volume_f,no_data = volDict(dirs, 50, vtype= 'zeros', rls=None, stcheck = 'Large_Diff')
# vol_f_6=pd.DataFrame.from_dict(volume_f, orient='index')

In [ ]:
# vol_f_6[['Systole RMSE', 'Diastole RMSE','EF RMSE']].sort_values('Diastole RMSE')
# vol_f_6['Vol_Method'] = 'zeros_Large_Diff'

In [14]:
tmp =pd.concat([vol_f.reset_index(),vol_f_1.reset_index(), vol_f_2.reset_index()]).reset_index()
#                 ,
#           vol_f_3.reset_index(), vol_f_4.reset_index(), vol_f_5.reset_index(),
#           vol_f_6.reset_index()]).reset_index()

In [15]:
tmp[['index','Vol Method','Systole RMSE','Diastole RMSE','EF RMSE']].sort_values('Systole RMSE')

,index,Vol Method,Systole RMSE,Diastole RMSE,EF RMSE
7,combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2,zeros__,12.40,16.93,8.0
12,combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2,zeros_rls_,12.44,14.97,10.0
9,combined_1_3_0_176_aug_drop_dice_crERDS2,zeros__,12.75,18.38,7.0
14,combined_1_3_0_176_aug_drop_dice_crERDS2,zeros_rls_,12.90,15.79,7.0
2,combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2,orig__,12.99,16.93,10.0
4,combined_1_3_0_176_aug_drop_dice_crERDS2,orig__,13.40,18.38,8.0
5,combined_1_3_0_176_CLAHE_augx_bce1_crERDS2,zeros__,13.58,16.34,9.0
10,combined_1_3_0_176_CLAHE_augx_bce1_crERDS2,zeros_rls_,13.69,14.82,9.0
6,combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2,zeros__,14.36,14.46,18.0
8,combined_1_3_0_176_CLAHE_augx_drop_ld1_crERDS2,zeros__,14.38,17.26,8.0


In [49]:
sys_path = '3combined_1_3_0_176_CLAHE_augx_drop_bce1_crERDS2' 
sys_v = vfw.computeVolume(v_dir+'1/'+sys_path+'/',176, vtype='zeros',STway='sub', ty=None, rls = None, stcheck = None)
dia_path = '3combined_1_3_0_176_CLAHE_augx_bn_bce1_crERDS2' 
dia_v = vfw.computeVolume(v_dir+'1/'+sys_path+'/',176, vtype='zeros',STway='sub', ty=None, rls = None, stcheck = None)
dt,dv,dia_df = vfw.create_df(dia_v)
st,sv,sys_df = vfw.create_df(sys_v)
dia_cols = ['Systole','Diastole','Age','Age_Int','Age Unit','Gender', 'Diastole_P']
sys_cols = ['Systole','Diastole','Age','Age_Int','Age Unit','Gender', 'Systole_P']
final_df = dia_df[dia_cols]
final_df['Systole_P'] = sys_df['Systole_P']
final_df['EF'] = (final_df['Diastole']-final_df['Systole'])/final_df['Diastole']
final_df['EF_P'] = (final_df['Diastole_P']-final_df['Systole_P'])/final_df['Diastole_P']
all_no_outliers = vfw.removeOutliers(final_df,dia_v, data=None)
print (all_no_outliers.columns)
test = vfw.removeLowEstimates(all_no_outliers)
print (test.columns)
all_no_out_noVol = vfw.removeNoVolume(test)
sys, dia, ef = vfw.compute_rmse(all_no_out_noVol, 'all')

Index(['Systole', 'Diastole', 'Age', 'Age_Int', 'Age Unit', 'Gender',
       'Diastole_P', 'Systole_P', 'EF', 'EF_P'],
      dtype='object')
Index(['Systole', 'Diastole', 'Age', 'Age_Int', 'Age Unit', 'Gender',
       'Diastole_P', 'Systole_P', 'EF', 'EF_P'],
      dtype='object')
Index(['Systole', 'Diastole', 'Age', 'Age_Int', 'Age Unit', 'Gender',
       'Diastole_P', 'Systole_P', 'EF', 'EF_P'],
      dtype='object')
all :  Systole RMSE: 12.40 ml  Diastole RMSE: '16.93 ml  EF RMSE: 8.00 %


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se